# Tutorial 03 - Hole

In [ ]:
import dolfinx.fem
import gmsh
import multiphenicsx.io
import multiphenicsx.mesh
import numpy as np
import petsc4py
import ufl

In [ ]:
import rbnicsx.backends

## 1. Mesh generation

In [ ]:
mesh_size = 1e-1

In [ ]:
subdomains_vertices = [
    [(-1.0, -1.0), (-2.0, -2.0), (1.0, -1.0)],  # subdomain 1
    [(1.0, -1.0), (-2.0, -2.0), (2.0, -2.0)],  # subdomain 2
    [(-1.0, -1.0), (-1.0, 1.0), (-2.0, -2.0)],  # subdomain 3
    [(-1.0, 1.0), (-2.0, 2.0), (-2.0, -2.0)],  # subdomain 4
    [(1.0, -1.0), (2.0, -2.0), (1.0, 1.0)],  # subdomain 5
    [(2.0, 2.0), (1.0, 1.0), (2.0, -2.0)],  # subdomain 6
    [(-1.0, 1.0), (1.0, 1.0), (-2.0, 2.0)],  # subdomain 7
    [(2.0, 2.0), (-2.0, 2.0), (1.0, 1.0)]  # subdomain 8
]

In [ ]:
boundaries_vertices = [
    [(-1.0, -1.0), (1.0, -1.0)],  # boundary 1, bottom inner
    [(-1.0, -1.0), (-1.0, 1.0)],  # boundary 2, left inner
    [(-1.0, 1.0), (1.0, 1.0)],  # boundary 3, top inner
    [(1.0, -1.0), (1.0, 1.0)],  # boundary 4, right inner
    [(-2.0, -2.0), (2.0, -2.0)],  # boundary 5, bottom outer
    [(-2.0, -2.0), (-2.0, 2.0)],  # boundary 6, left outer
    [(-2.0, 2.0), (2.0, 2.0)],  # boundary 7, top outer
    [(2.0, -2.0), (2.0, 2.0)]  # boundary 8, right outer
]

In [ ]:
gmsh.initialize()
gmsh.model.add("hole")

In [ ]:
gmsh_points = dict()
for subdomain_vertices in subdomains_vertices:
    for vertex in subdomain_vertices:
        try:
            gmsh_points[vertex]
        except KeyError:
            gmsh_points[vertex] = gmsh.model.geo.addPoint(vertex[0], vertex[1], 0.0, mesh_size)

In [ ]:
gmsh_lines = dict()
gmsh_subdomains = list()
for subdomain_vertices in subdomains_vertices:
    subdomain_lines = list()
    for v in range(3):
        key = (gmsh_points[subdomain_vertices[v]], gmsh_points[subdomain_vertices[(v + 1) % 3]])
        try:
            gmsh_lines[key]
        except KeyError:
            gmsh_lines[key] = gmsh.model.geo.addLine(*key)
            gmsh_lines[key[1], key[0]] = - gmsh_lines[key]
        subdomain_lines.append(gmsh_lines[key])
    subdomain_lines_loop = gmsh.model.geo.addCurveLoop(subdomain_lines)
    gmsh_subdomains.append(gmsh.model.geo.addPlaneSurface([subdomain_lines_loop]))

In [ ]:
gmsh.model.geo.synchronize()
for (label, gmsh_subdomain) in enumerate(gmsh_subdomains):
    gmsh.model.addPhysicalGroup(2, [gmsh_subdomain], label + 1)
for (label, boundary) in enumerate(boundaries_vertices):
    gmsh.model.addPhysicalGroup(1, [gmsh_lines[gmsh_points[boundary[0]], gmsh_points[boundary[1]]]], label + 1)
gmsh.model.mesh.generate(2)

In [ ]:
mesh, subdomains, boundaries = multiphenicsx.mesh.gmsh_to_fenicsx(gmsh.model, gdim=2)
gmsh.finalize()

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

In [ ]:
multiphenicsx.io.plot_mesh_tags(subdomains)

In [ ]:
multiphenicsx.io.plot_mesh_tags(boundaries)

## 2. Problem definition

In [ ]:
class HarmonicExtension(rbnicsx.backends.MeshMotion):
    """Extend the shape parametrization from the boundary to the interior with an harmonic extension."""

    def __init__(self, mu: np.typing.NDArray[np.float64]) -> None:
        # Define function space
        M = dolfinx.fem.VectorFunctionSpace(mesh, ("Lagrange", mesh.geometry.cmap.degree))
        # Define trial and test functions
        m = ufl.TrialFunction(M)
        n = ufl.TestFunction(M)
        # Define bilinear form of the harmonic extension problem
        a_he = dolfinx.fem.form(ufl.inner(ufl.grad(m), ufl.grad(n)) * ufl.dx)
        a_he_cpp = dolfinx.fem.form(a_he)
        # Define linear form of the harmonic extension problem
        zero_vector = dolfinx.fem.Constant(mesh, np.zeros(mesh.topology.dim, petsc4py.PETSc.ScalarType))
        f_he = dolfinx.fem.form(ufl.inner(zero_vector, n) * ufl.dx)
        f_he_cpp = dolfinx.fem.form(f_he)
        # Define boundary conditions for the harmonic extension problem
        facets_inner_boundaries = boundaries.indices[np.isin(boundaries.values, (1, 2, 3, 4))]
        facets_outer_boundaries = boundaries.indices[np.isin(boundaries.values, (5, 6, 7, 8))]
        bdofs_M_inner_boundaries = dolfinx.fem.locate_dofs_topological(
            M, mesh.topology.dim - 1, facets_inner_boundaries)
        bdofs_M_outer_boundaries = dolfinx.fem.locate_dofs_topological(
            M, mesh.topology.dim - 1, facets_outer_boundaries)
        shape_parametrization_inner_boundaries = dolfinx.fem.Function(M)
        shape_parametrization_inner_boundaries.interpolate(lambda x: (mu[0] * x[0], mu[1] * x[1]))
        shape_parametrization_outer_boundaries = dolfinx.fem.Function(M)
        shape_parametrization_outer_boundaries.interpolate(lambda x: (x[0], x[1]))
        bcs_he = [
            dolfinx.fem.dirichletbc(shape_parametrization_inner_boundaries, bdofs_M_inner_boundaries),
            dolfinx.fem.dirichletbc(shape_parametrization_outer_boundaries, bdofs_M_outer_boundaries)]
        # Assemble the left-hand side matrix of the harmonic extension problem
        A = dolfinx.fem.assemble_matrix(a_he_cpp, bcs=bcs_he)
        A.assemble()
        # Assemble the right-hand side vector of the harmonic extension problem
        F = dolfinx.fem.assemble_vector(f_he_cpp)
        dolfinx.fem.apply_lifting(F, [a_he_cpp], [bcs_he])
        F.ghostUpdate(addv=petsc4py.PETSc.InsertMode.ADD, mode=petsc4py.PETSc.ScatterMode.REVERSE)
        dolfinx.fem.set_bc(F, bcs_he)
        # Solve the harmonic extension problem
        ksp = petsc4py.PETSc.KSP()
        ksp.create(mesh.comm)
        ksp.setOperators(A)
        ksp.setType("preonly")
        ksp.getPC().setType("lu")
        ksp.getPC().setFactorSolverType("mumps")
        ksp.setFromOptions()
        shape_parametrization = dolfinx.fem.Function(M)
        ksp.solve(F, shape_parametrization.vector)
        shape_parametrization.vector.ghostUpdate(
            addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
        # Initialize mesh motion object
        super().__init__(mesh, shape_parametrization)

In [ ]:
mu_mesh_motion = np.array([0.5, 0.5, np.nan])
with HarmonicExtension(mu_mesh_motion) as harmonic_extension:
    multiphenicsx.io.plot_mesh(mesh)

In [ ]:
multiphenicsx.io.plot_vector_field(harmonic_extension.deformation, "domain deformation", glyph_factor=1.0)

In [ ]:
del harmonic_extension

In [ ]:
class Problem(object):
    """Define a linear problem, and solve it with KSP."""

    def __init__(self) -> None:
        # Define function space
        V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 1))
        self._V = V
        # Define trial and test functions
        u = ufl.TrialFunction(V)
        v = ufl.TestFunction(V)
        # Define measures for integration of forms
        dx = ufl.Measure("dx")(subdomain_data=subdomains)
        ds = ufl.Measure("ds")(subdomain_data=boundaries)
        # Define symbolic parameters for use in UFL forms
        mu_symb = rbnicsx.backends.SymbolicParameters(mesh, shape=(3, ))
        self._mu_symb = mu_symb
        # Define bilinear form of the problem
        a = (
            ufl.inner(ufl.grad(u), ufl.grad(v)) * dx + mu_symb[2] * (
                ufl.inner(u, v) * ds(5) + ufl.inner(u, v) * ds(6)
                + ufl.inner(u, v) * ds(7) + ufl.inner(u, v) * ds(8)
            )
        )
        self._a = a
        self._a_cpp = dolfinx.fem.form(a)
        # Define linear form of the problem
        one = petsc4py.PETSc.ScalarType(1)
        f = (
            ufl.inner(one, v) * ds(1) + ufl.inner(one, v) * ds(2)
            + ufl.inner(one, v) * ds(3) + ufl.inner(one, v) * ds(4)
        )
        self._f = f
        self._f_cpp = dolfinx.fem.form(f)
        # Prepare storage for mesh motion object
        self._mesh_motion = None

    @property
    def function_space(self) -> dolfinx.fem.FunctionSpace:
        """Return the function space of the problem."""
        return self._V

    @property
    def bilinear_form(self) -> ufl.Form:
        """Return the bilinear form of the problem."""
        return self._a

    @property
    def linear_form(self) -> ufl.Form:
        """Return the linear form of the problem."""
        return self._f

    @property
    def mesh_motion(self) -> rbnicsx.backends.MeshMotion:
        """Return the mesh motion object that was used in the latest solve."""
        return self._mesh_motion

    def _assemble_matrix(self) -> petsc4py.PETSc.Mat:
        """Assemble the left-hand side matrix."""
        A = dolfinx.fem.assemble_matrix(self._a_cpp)
        A.assemble()
        return A

    def _assemble_vector(self) -> petsc4py.PETSc.Vec:
        """Assemble the right-hand side vector."""
        F = dolfinx.fem.assemble_vector(self._f_cpp)
        F.ghostUpdate(addv=petsc4py.PETSc.InsertMode.ADD, mode=petsc4py.PETSc.ScatterMode.REVERSE)
        return F

    def solve(self, mu: np.typing.NDArray[np.float64]) -> dolfinx.fem.Function:
        """Assign the provided parameters value, apply shape parametrization and solve the problem."""
        self._mu_symb.value[:] = mu
        with HarmonicExtension(mu) as self._mesh_motion:
            return self._solve()

    def _solve(self) -> dolfinx.fem.Function:
        """Solve the linear problem with KSP."""
        A = self._assemble_matrix()
        F = self._assemble_vector()
        ksp = petsc4py.PETSc.KSP()
        ksp.create(mesh.comm)
        ksp.setOperators(A)
        ksp.setType("preonly")
        ksp.getPC().setType("lu")
        ksp.getPC().setFactorSolverType("mumps")
        ksp.setFromOptions()
        solution = dolfinx.fem.Function(self._V)
        ksp.solve(F, solution.vector)
        solution.vector.ghostUpdate(
            addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
        return solution

In [ ]:
problem = Problem()

In [ ]:
mu_solve = np.array([0.5, 0.5, 0.01])
solution = problem.solve(mu_solve)

In [ ]:
multiphenicsx.io.plot_scalar_field(solution, "high fidelity solution on the reference domain")

In [ ]:
with problem.mesh_motion:
    multiphenicsx.io.plot_scalar_field(solution, "high fidelity solution on the deformed domain")